In [ ]:
import tensorflow as tf
import os #for file management
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_dir = r'/content/drive/MyDrive/Final Train Data/Leaf' #  dataset directory

In [ ]:

IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,

    # data augmentation
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=30,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)


Found 1236 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Found 308 images belonging to 4 classes.
9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
## labels
print(train_generator.class_indices) 
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('leaves.txt', 'w') as f: 
    f.write(labels)

In [ ]:

# get the first batch of images
x_batch, y_batch = next(train_generator)

# get the class names from the generator
class_names = list(train_generator.class_indices.keys())

# plot the first 9 images in the batch
plt.figure(figsize=(10, 10))
for i in range(12):
    # get the true class of the image
    true_class = class_names[y_batch[i].argmax()]

    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(x_batch[i])
    plt.title(true_class)
    plt.axis("off")
plt.show()


## Transfer Learning Concept


In [ ]:
## Model
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3) 

## MobileNet V2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:

## Tuned up model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 conv2d (Conv2D)             (None, 5, 5, 64)          737344    
                                                                 
 batch_normalization (BatchN  (None, 5, 5, 64)         256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 64)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 2, 2, 64)          0         
                                                        

In [ ]:
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        print(layer.kernel_size)

(3, 3)


In [ ]:
epochs = 10 
history = model.fit(
    train_generator, 
    epochs = epochs, 
    validation_data=val_generator
)

Epoch 1/10
20/20 [==============================] - 111s 5s/step - loss: 0.7646 - accuracy: 0.7387 - precision: 0.7676 - recall: 0.6893 - val_loss: 0.3855 - val_accuracy: 0.9058 - val_precision: 0.9148 - val_recall: 0.9058
Epoch 2/10
20/20 [==============================] - 124s 6s/step - loss: 0.2350 - accuracy: 0.9110 - precision: 0.9271 - recall: 0.8948 - val_loss: 0.0742 - val_accuracy: 0.9740 - val_precision: 0.9803 - val_recall: 0.9708
Epoch 3/10
20/20 [==============================] - 102s 5s/step - loss: 0.1498 - accuracy: 0.9474 - precision: 0.9547 - recall: 0.9377 - val_loss: 0.1904 - val_accuracy: 0.9481 - val_precision: 0.9481 - val_recall: 0.9481
Epoch 4/10
20/20 [==============================] - 102s 5s/step - loss: 0.1090 - accuracy: 0.9684 - precision: 0.9699 - recall: 0.9636 - val_loss: 0.0893 - val_accuracy: 0.9740 - val_precision: 0.9739 - val_recall: 0.9675
Epoch 5/10
20/20 [==============================] - 102s 5s/step - loss: 0.0918 - accuracy: 0.9709 - precisi

In [ ]:
## Converting the model to tflite
saved_model_dir = '' 
tf.saved_model.save(model, saved_model_dir) 
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) 
tflite_model = converter.convert()
with open('pest.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
## Saving the file
from google.colab import files
files.download('pest.tflite')
files.download('pest.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Analyzing the Result

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
precision = history.history['precision']
val_precision = history.history['val_precision']
recall = history.history['recall']
val_recall = history.history['val_recall']

In [ ]:
print(acc)
print(val_acc)
print(loss)
print(val_loss)
print(precision)
print(val_precision)
print(recall)
print(val_recall )

[0.7386731505393982, 0.9110032320022583, 0.9474110007286072, 0.9684466123580933, 0.9708737730979919, 0.9635922312736511, 0.9733009934425354, 0.9652103781700134, 0.983818769454956, 0.9765372276306152]
[0.9058441519737244, 0.9740259647369385, 0.948051929473877, 0.9740259647369385, 0.9675324559211731, 0.9772727489471436, 0.9577922224998474, 0.9772727489471436, 0.948051929473877, 0.9902597665786743]
[0.7646228075027466, 0.2350025326013565, 0.14977362751960754, 0.10896081477403641, 0.09178616851568222, 0.10551943629980087, 0.07348357141017914, 0.10019132494926453, 0.058571554720401764, 0.06954573094844818]
[0.38554492592811584, 0.0742066279053688, 0.19039426743984222, 0.08925361931324005, 0.08511090278625488, 0.06612588465213776, 0.17099256813526154, 0.07956206053495407, 0.16313867270946503, 0.04251027852296829]
[0.7675675749778748, 0.9270746111869812, 0.9546952247619629, 0.9698697328567505, 0.9747557044029236, 0.9649837017059326, 0.974047064781189, 0.9667747020721436, 0.985401451587677, 0.

In [ ]:
final_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
final_precision = history.history['precision'][-1]
final_val_precision = history.history['val_precision'][-1]
final_recall = history.history['recall'][-1]
final_val_recall = history.history['val_recall'][-1]

In [ ]:
print("final_acc:",final_acc)
print("final_val_acc:",final_val_acc)
print("final_loss:",final_loss)
print("final_val_loss:", final_val_loss)
print("final_precision:" ,final_precision)
print("final_val_precision:",final_val_precision)
print("final_recall: ",final_recall)
print("final_recall: ", final_val_recall)

final_acc: 0.9765372276306152
final_val_acc: 0.9902597665786743
final_loss: 0.06954573094844818
final_val_loss: 0.04251027852296829
final_precision: 0.978066623210907
final_val_precision:  0.990227997303009
final_recall:  0.9741100072860718
final_recall:  0.9870129823684692
